In [138]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.engine.saving import load_model
from tensorflow.python.keras.layers import Dense


class LossHistory(keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))


def autoencoder():
    input_image = Input(shape=(784,))
    # Encoder
    encoder = Dense(units=784, activation='relu')(input_image)
    encoder = Dense(units=512, activation='relu')(encoder)
    encoder = Dense(units=256, activation='relu')(encoder)
    encoder = Dense(units=128, activation='relu')(encoder)
    encoder = Dense(units=64, activation='relu')(encoder)
    encoder = Dense(units=32, activation='relu')(encoder)

    # Decoder
    decoder = Dense(units=64, activation='relu')(encoder)
    decoder = Dense(units=128, activation='relu')(decoder)
    decoder = Dense(units=256, activation='relu')(decoder)
    decoder = Dense(units=512, activation='relu')(decoder)
    decoder = Dense(units=784, activation='sigmoid')(decoder)

    enc = Model(
        input_image, encoder
    )

    autoenc = Model(
        input_image, decoder
    )

    autoenc.compile(loss='binary_crossentropy', optimizer='adam')

    return enc, autoenc


def trained_encoder():
    """Return a keras model"""
    return load_model("encoder_model", compile=True)


def trained_autoencoder():
    return load_model("autoencoder_model")


def generate_trained_models(epochs):
    (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
    x_train = np.reshape(x_train, (60000, 784))
    x_test = np.reshape(x_test, (10000, 784))
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    encoder, autoenc = autoencoder()
    lossHistory = LossHistory()
    #autoenc.summary()
    nbr_sample=5000
    autoenc.fit(x_train[0:nbr_sample], x_train[0:nbr_sample], epochs=epochs, batch_size=128, shuffle=True, validation_data=(x_test[0:nbr_sample], x_test[0:nbr_sample]),
                callbacks=[lossHistory])


    encoder.save("encoder_model")
    autoenc.save("autoencoder_model")
    encoded_images = trained_encoder().predict(x_train)
    


def generate_graphs():
    _, (x_test, _) = keras.datasets.fashion_mnist.load_data()
    x_test = np.reshape(x_test, (10000, 784))
    x_test = x_test.astype('float32') / 255
    autoencoded_images = trained_autoencoder().predict(x_test[0:10]).reshape((10, 28, 28))
    for i in range(0, 10):
        figure = plt.figure()
        im1 = figure.add_subplot(2, 1, 1)
        im1.imshow(x_test[i].reshape((28, 28)) * 255)
        im2 = figure.add_subplot(2, 1, 2)
        im2.imshow(autoencoded_images[i].reshape((28, 28)) * 255)
        plt.savefig("images/img" + str(i))
        plt.clf()

In [2]:
from tensorflow import keras
def get_encoded_data():
    model = trained_encoder()
    (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
    x_train = x_train.reshape((60000, 784))
    x_train = x_train.astype('float32') / 255
    x_reshaped = x_test.reshape((10000, 784))
    x_reshaped = x_reshaped.astype('float32') / 255
    x_train_encoded = model.predict(x_train)
    x_test_encoded = model.predict(x_reshaped)

    return (x_train_encoded, y_train), (x_test_encoded, y_test)

In [3]:
import numpy as np
from scipy.stats import mode
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from tensorflow import keras



def fashion_scatter(x, colors):
    import seaborn as sns
    import matplotlib.pyplot as plt
    import matplotlib.patheffects as PathEffects
    # on choisit une palette de de couleur avec seaborn
    num_classes = len(np.unique(colors))
    palette = np.array(sns.color_palette("hls", num_classes))

    # on crée le scatter-plot
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:, 0], x[:, 1], lw=0, s=40, c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # on ajoute les labels pour chaque digit correspondant au label
    txts = []

    for i in range(num_classes):
        # position de chaque label au milieu des points

        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts


def kmeans_fit():
    import time
    from sklearn.metrics.cluster import adjusted_mutual_info_score
    from sklearn.metrics.cluster import normalized_mutual_info_score
    (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
    (x_train_encoded, y_train), (x_test_encoded, y_train) = get_encoded_data()
    
    #encoded data
    k_means = KMeans(n_clusters=10, random_state=0)
    time_start = time.time()
    k_means.fit(x_train_encoded)
    print('K-means done! Time elapsed: {} seconds'.format(time.time() - time_start))
    test_clusters = k_means.predict(x_test_encoded)

    labels = np.zeros_like(test_clusters)
    for i in range(10):
        mask = (test_clusters == i)
        labels[mask] = mode(y_test[mask])[0]
        print("nombre de : ", i, " ", len(test_clusters[test_clusters == i]))

    cent = k_means.cluster_centers_
    score_ami = adjusted_mutual_info_score(labels, test_clusters)
    score_nmi = normalized_mutual_info_score(labels, test_clusters)
    print("ami k-means encoded data: ", score_ami)
    print("nmi k-means encoded data: ", score_nmi)
    



def tsne(per):
    import time
    from sklearn.manifold import TSNE
    (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
    (x_train_encoded, y_train), (x_test_encoded, y_test) = get_encoded_data()
    time_start = time.time()
    print('encoded data')
    tsne = TSNE(n_components=2, verbose=1, perplexity=per, n_iter=250)
    tsne_results = tsne.fit_transform(x_train_encoded)
    figure, _, _, _ = fashion_scatter(tsne_results, y_train)
    figure.show()
    print('t-SNE done! Time elapsed: {} seconds'.format(time.time() - time_start))

In [16]:
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import normalized_mutual_info_score
def cluster_ensemble(epochs):
    nbr_sample=10000
    Co_Ass = [[] for x in range(nbr_sample)]
    part=[[],[],[],[],[]]
    i=0
    for ep in epochs:
        generate_trained_models(ep)
        (x_train_encoded, y_train), (x_test_encoded, y_test) = get_encoded_data()
        k_means = KMeans(n_clusters=10, random_state=0)
        k_means.fit(x_train_encoded[0:nbr_sample])
        test_clusters = k_means.predict(x_train_encoded[0:nbr_sample])
        #labels = np.zeros_like(test_clusters)
        score_ami = adjusted_mutual_info_score(y_train[0:nbr_sample], test_clusters)
        score_nmi = normalized_mutual_info_score(y_train[0:nbr_sample], test_clusters)
        print("number of epoch : ",ep)
        print("ami k-means for non encoded data: ", score_ami)
        print("nmi k-means for non encoded data: ", score_nmi)
        part[i]=test_clusters
        print(len(part[i]))
        i=i+1

    
    for i in range(0,nbr_sample):
        for j in range(0,nbr_sample):
            for k in range(0,len(epochs)):
                som=0
                if(part[k][i]==part[k][j]):
                    som=som+1
                som=som/5.0
                Co_Ass[i].append(som)
    k_means = KMeans(n_clusters=10, random_state=0)
    k_means.fit(Co_Ass)
    test_clusters = k_means.predict(Co_Ass)
    score_ami = adjusted_mutual_info_score(y_train[0:nbr_sample], test_clusters)
    score_nmi = normalized_mutual_info_score(y_train[0:nbr_sample], test_clusters)
    
    print("ami k-means for CO-assosiation matrix: ", score_ami)
    print("nmi k-means for CO-assosiation matrix: ", score_nmi)
          
          


For this last question we inspired from the research paper Spectral Clustering via ensemble Deep AutoencoderLearning (SC-EDAE), but only for the first step where we use multiple autoencoder on our input data.
These autoencoder are generated either by changing the numbers of epochs and initialization or by changing the architecture of the encoder model, in our experimentation we will simply modify the number of epochs and the initialization without modifying the architecture (changing the model's architecture can be a interresting improvment to make for futur experimentations).<br>
we choosed five autoencoder with respectively 50,75,100,125 and 150 epochs in this order the model used is the same as for the previous questions, the input data pass through the 5 antoencoder in parallel (just the encoder part of the autoencoder the decoder part is not of any use in this case), we get our five matrix Y1 Y2 Y3 Y4 and Y5, it's from this moment that we follow a different path from the research paper.<br>
We apply a K-means on each Y matrix to have 5 different clustering let's call thel C1 C2 C3 C4 C5, after try a consensus clustering alogorithm find in another research paper "A survey of clustering ensemble algorithms", the method is named Co-Association matrix based methods, this method allow us to avoid the label correspondence problem.
This method map the partitions in the cluster into an antermediate representation : the co association matrix, the only problem with this methid can be the computational time although it only needs to be computed once.<br>
After creating this matrix the last step is to apply another K-means on this Co-Association matrix. <br>
**note : we took only 10000 balanced samples on the 60000 samples for computational reason**

In [14]:
epochs=[50,75,100,125,150]
cluster_ensemble(epochs)

Train on 5000 samples, validate on 5000 samples
Epoch 1/50
5000/5000 [==============================] - 9s 2ms/sample - loss: 0.5172 - val_loss: 0.4444
Epoch 2/50
5000/5000 [==============================] - 1s 234us/sample - loss: 0.4037 - val_loss: 0.3926
Epoch 3/50
5000/5000 [==============================] - 1s 233us/sample - loss: 0.3683 - val_loss: 0.3593
Epoch 4/50
5000/5000 [==============================] - 1s 230us/sample - loss: 0.3517 - val_loss: 0.3499
Epoch 5/50
5000/5000 [==============================] - 1s 224us/sample - loss: 0.3394 - val_loss: 0.3383
Epoch 6/50
5000/5000 [==============================] - 1s 289us/sample - loss: 0.3314 - val_loss: 0.3336
Epoch 7/50
5000/5000 [==============================] - 1s 253us/sample - loss: 0.3261 - val_loss: 0.3366
Epoch 8/50
5000/5000 [==============================] - 2s 372us/sample - loss: 0.3244 - val_loss: 0.3272
Epoch 9/50
5000/5000 [==============================] - 2s 385us/sample - loss: 0.3186 - val_loss: 0.3247


D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


number of epoch :  50
ami k-means for non encoded data:  0.352133790274631
nmi k-means for non encoded data:  0.38293888240422674
5000
Train on 5000 samples, validate on 5000 samples
Epoch 1/75
5000/5000 [==============================] - 4s 781us/sample - loss: 0.5060 - val_loss: 0.4195
Epoch 2/75
5000/5000 [==============================] - 1s 232us/sample - loss: 0.3969 - val_loss: 0.3936
Epoch 3/75
5000/5000 [==============================] - 1s 236us/sample - loss: 0.3643 - val_loss: 0.3566
Epoch 4/75
5000/5000 [==============================] - 1s 229us/sample - loss: 0.3466 - val_loss: 0.3474
Epoch 5/75
5000/5000 [==============================] - 1s 237us/sample - loss: 0.3347 - val_loss: 0.3358
Epoch 6/75
5000/5000 [==============================] - 1s 245us/sample - loss: 0.3268 - val_loss: 0.3344
Epoch 7/75
5000/5000 [==============================] - 1s 224us/sample - loss: 0.3235 - val_loss: 0.3285
Epoch 8/75
5000/5000 [==============================] - 1s 222us/sample - l

D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


number of epoch :  75
ami k-means for non encoded data:  0.39742438669112956
nmi k-means for non encoded data:  0.41953643084833747
5000
Train on 5000 samples, validate on 5000 samples
Epoch 1/100
5000/5000 [==============================] - 4s 875us/sample - loss: 0.4971 - val_loss: 0.4172
Epoch 2/100
5000/5000 [==============================] - 1s 237us/sample - loss: 0.3939 - val_loss: 0.3814
Epoch 3/100
5000/5000 [==============================] - 1s 238us/sample - loss: 0.3690 - val_loss: 0.3626
Epoch 4/100
5000/5000 [==============================] - 1s 252us/sample - loss: 0.3527 - val_loss: 0.3613
Epoch 5/100
5000/5000 [==============================] - 2s 308us/sample - loss: 0.3391 - val_loss: 0.3412
Epoch 6/100
5000/5000 [==============================] - 2s 327us/sample - loss: 0.3319 - val_loss: 0.3291
Epoch 7/100
5000/5000 [==============================] - 1s 243us/sample - loss: 0.3196 - val_loss: 0.3236
Epoch 8/100
5000/5000 [==============================] - 1s 260us/

...

D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


number of epoch :  150
ami k-means for non encoded data:  0.40448900250050573
nmi k-means for non encoded data:  0.4244923724347774
5000


D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


ami k-means for CO-assosiation matrix:  0.3759632687014525
ami k-means for CO-assosiation matrix:  0.3870366012026818


D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


| epoch | ami | nmi |
| --- | --- | --- |
| 50 | 0.352 | 0.382|
| 75 | 0.397 |  0.419|
| 100 | 0.399 |  0.421|
| 125 | 0.395 |  0.416|
| 150 | 0.404 |  0.424|

Since applying the k-means on the Co-Association matrix gave 0.37 in term of ami and 0.38 in term of nmi , we can conclude that applying many autoencoder and then creating a Co-association matrix as cluster ensemble, give results similar as applying K-means directly on the encoded data.

**Let's try to improve the number of epochs to see if the results will improve.**

In [17]:
epochs=[150,200,250]
cluster_ensemble(epochs)

Train on 1000 samples, validate on 1000 samples
Epoch 1/150
1000/1000 [==============================] - 13s 13ms/sample - loss: 0.6388 - val_loss: 0.5379
Epoch 2/150
1000/1000 [==============================] - 1s 571us/sample - loss: 0.5148 - val_loss: 0.4938
Epoch 3/150
1000/1000 [==============================] - 1s 587us/sample - loss: 0.4849 - val_loss: 0.4757
Epoch 4/150
1000/1000 [==============================] - 1s 675us/sample - loss: 0.4594 - val_loss: 0.4395
Epoch 5/150
1000/1000 [==============================] - 1s 614us/sample - loss: 0.4252 - val_loss: 0.4172
Epoch 6/150
1000/1000 [==============================] - 0s 469us/sample - loss: 0.4074 - val_loss: 0.4047
Epoch 7/150
1000/1000 [==============================] - 1s 554us/sample - loss: 0.3925 - val_loss: 0.3939
Epoch 8/150
1000/1000 [==============================] - 1s 688us/sample - loss: 0.3887 - val_loss: 0.3857
Epoch 9/150
1000/1000 [==============================] - 1s 701us/sample - loss: 0.3781 - val_lo

D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


number of epoch :  150
ami k-means for non encoded data:  0.3723171400555771
nmi k-means for non encoded data:  0.4010383090764915
1000
Train on 1000 samples, validate on 1000 samples
Epoch 1/200
1000/1000 [==============================] - 7s 7ms/sample - loss: 0.6434 - val_loss: 0.5426
Epoch 2/200
1000/1000 [==============================] - 0s 312us/sample - loss: 0.5190 - val_loss: 0.4989
Epoch 3/200
1000/1000 [==============================] - 0s 336us/sample - loss: 0.4883 - val_loss: 0.4829
Epoch 4/200
1000/1000 [==============================] - 0s 259us/sample - loss: 0.4699 - val_loss: 0.4549
Epoch 5/200
1000/1000 [==============================] - 0s 266us/sample - loss: 0.4342 - val_loss: 0.4340
Epoch 6/200
1000/1000 [==============================] - 1s 632us/sample - loss: 0.4172 - val_loss: 0.4189
Epoch 7/200
1000/1000 [==============================] - 0s 318us/sample - loss: 0.4064 - val_loss: 0.4114
Epoch 8/200
1000/1000 [==============================] - 0s 327us/sam

1000/1000 [==============================] - 0s 271us/sample - loss: 0.2774 - val_loss: 0.3112
Epoch 151/200
1000/1000 [==============================] - 0s 258us/sample - loss: 0.2775 - val_loss: 0.3128
Epoch 152/200
1000/1000 [==============================] - 0s 264us/sample - loss: 0.2780 - val_loss: 0.3131
Epoch 153/200
1000/1000 [==============================] - 0s 274us/sample - loss: 0.2789 - val_loss: 0.3123
Epoch 154/200
1000/1000 [==============================] - 0s 251us/sample - loss: 0.2785 - val_loss: 0.3134
Epoch 155/200
1000/1000 [==============================] - 0s 265us/sample - loss: 0.2786 - val_loss: 0.3113
Epoch 156/200
1000/1000 [==============================] - 0s 260us/sample - loss: 0.2777 - val_loss: 0.3117
Epoch 157/200
1000/1000 [==============================] - 0s 262us/sample - loss: 0.2771 - val_loss: 0.3116
Epoch 158/200
1000/1000 [==============================] - 0s 281us/sample - loss: 0.2774 - val_loss: 0.3124
Epoch 159/200
1000/1000 [========

D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


...

1000/1000 [==============================] - 0s 472us/sample - loss: 0.2761 - val_loss: 0.3086
Epoch 151/250
1000/1000 [==============================] - 0s 349us/sample - loss: 0.2764 - val_loss: 0.3076
Epoch 152/250
1000/1000 [==============================] - 0s 467us/sample - loss: 0.2761 - val_loss: 0.3079
Epoch 153/250
1000/1000 [==============================] - 1s 538us/sample - loss: 0.2759 - val_loss: 0.3080
Epoch 154/250
1000/1000 [==============================] - 0s 277us/sample - loss: 0.2760 - val_loss: 0.3081
Epoch 155/250
1000/1000 [==============================] - 0s 282us/sample - loss: 0.2760 - val_loss: 0.3082
Epoch 156/250
1000/1000 [==============================] - 0s 300us/sample - loss: 0.2766 - val_loss: 0.3079
Epoch 157/250
1000/1000 [==============================] - 0s 272us/sample - loss: 0.2772 - val_loss: 0.3102
Epoch 158/250
1000/1000 [==============================] - 0s 323us/sample - loss: 0.2772 - val_loss: 0.3098
Epoch 159/250
1000/1000 [========

1000/1000 [==============================] - 0s 291us/sample - loss: 0.2675 - val_loss: 0.3095
Epoch 226/250
1000/1000 [==============================] - 0s 286us/sample - loss: 0.2674 - val_loss: 0.3095
Epoch 227/250
1000/1000 [==============================] - 0s 313us/sample - loss: 0.2674 - val_loss: 0.3098
Epoch 228/250
1000/1000 [==============================] - 0s 321us/sample - loss: 0.2675 - val_loss: 0.3094
Epoch 229/250
1000/1000 [==============================] - 0s 352us/sample - loss: 0.2675 - val_loss: 0.3097
Epoch 230/250
1000/1000 [==============================] - 0s 301us/sample - loss: 0.2677 - val_loss: 0.3098
Epoch 231/250
1000/1000 [==============================] - 0s 263us/sample - loss: 0.2680 - val_loss: 0.3112
Epoch 232/250
1000/1000 [==============================] - ETA: 0s - loss: 0.267 - 0s 269us/sample - loss: 0.2680 - val_loss: 0.3096
Epoch 233/250
1000/1000 [==============================] - 0s 262us/sample - loss: 0.2675 - val_loss: 0.3098
Epoch 234

D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


number of epoch :  250
ami k-means for non encoded data:  0.3967545939348004
nmi k-means for non encoded data:  0.4355662380244354
1000
ami k-means for CO-assosiation matrix:  0.37116572176235013
ami k-means for CO-assosiation matrix:  0.39778563862253363


D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


| epoch | ami | nmi |
| --- | --- | --- |
| 150 | 0.372 | 0.401|
| 200 | 0.402 |  0.443|
| 250 | 0.396 |  0.435|


**Improving the number of epochs didn't change  anything, the results are even worse than the worst tentative ( with 150 epoch)**

In [137]:
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import normalized_mutual_info_score
from collections import Counter
def vote_ensemble(epochs):
    nbr_sample=5000
    #Co_Ass = [[] for x in range(nbr_sample)]
    part=[[],[],[],[],[],[],[],[],[],[]]
    i=0
    for ep in epochs:
        generate_trained_models(ep)
        (x_train_encoded, y_train), (x_test_encoded, y_test) = get_encoded_data()
        k_means = KMeans(n_clusters=10, random_state=0)
        k_means.fit(x_train_encoded[0:nbr_sample])
        test_clusters = k_means.predict(x_train_encoded[0:nbr_sample])
        #labels = np.zeros_like(test_clusters)
        score_ami = adjusted_mutual_info_score(y_train[0:nbr_sample], test_clusters)
        score_nmi = normalized_mutual_info_score(y_train[0:nbr_sample], test_clusters)
        print("number of epoch : ",ep)
        print("ami k-means for non encoded data: ", score_ami)
        print("nmi k-means for non encoded data: ", score_nmi)
        part[i]=test_clusters
        #print(len(part[i]))
        i=i+1
    r=0
    part_final=[]
    for i in range(0,nbr_sample):
        comp=[]
        for k in range(0,len(epochs)):
            comp.append(part[k][i]) 
            #print(part[k][i])
        data = Counter(comp)
       
        #print(cmp)
        #print(data)
        if(len(data.most_common())>1):
            if(data.most_common(2)[1][1]==data.most_common(2)[0][1]):
                if(r%2==0):
                    part_final.append(data.most_common(2)[1][0])
                else:
                    part_final.append(data.most_common(2)[0][0]) 
                r=r+1
            else:
                part_final.append(data.most_common(1)[0][0]) 
        else:
            part_final.append(data.most_common(1)[0][0])
        
    score_ami = adjusted_mutual_info_score(y_train[0:nbr_sample], part_final)
    score_nmi = normalized_mutual_info_score(y_train[0:nbr_sample], part_final)
    print("ami for voting system: ", score_ami)
    print("nmo for voting system: ", score_nmi)

This time we'll keep our five autoencoder as before, we keep also the number of epoch for each autoencoder, but this time instead of creating the Co-assosiation matrix we will use a voting system that's mean if for exemple we have a input data xi, if 3 autoencoder vote for the label number 5 and the two others vote for another label, we'll map xi to the label number 5.<br>
If there is a equality in the vote we choose the label randomly

In [139]:
epochs=[50,75,100,125,150]
vote_ensemble(epochs)

Train on 5000 samples, validate on 5000 samples
Epoch 1/50
5000/5000 [==============================] - 27s 5ms/sample - loss: 0.5259 - val_loss: 0.4517
Epoch 2/50
5000/5000 [==============================] - 2s 334us/sample - loss: 0.4075 - val_loss: 0.3929
Epoch 3/50
5000/5000 [==============================] - 2s 405us/sample - loss: 0.3694 - val_loss: 0.3634
Epoch 4/50
5000/5000 [==============================] - 2s 337us/sample - loss: 0.3515 - val_loss: 0.3545
Epoch 5/50
5000/5000 [==============================] - 2s 306us/sample - loss: 0.3432 - val_loss: 0.3471
Epoch 6/50
5000/5000 [==============================] - 2s 387us/sample - loss: 0.3342 - val_loss: 0.3415
Epoch 7/50
5000/5000 [==============================] - 2s 312us/sample - loss: 0.3295 - val_loss: 0.3359
Epoch 8/50
5000/5000 [==============================] - 1s 294us/sample - loss: 0.3248 - val_loss: 0.3265
Epoch 9/50
5000/5000 [==============================] - 2s 332us/sample - loss: 0.3178 - val_loss: 0.3254

D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


number of epoch :  50
ami k-means for non encoded data:  0.37001347785126293
nmi k-means for non encoded data:  0.3986185242288014
Train on 5000 samples, validate on 5000 samples
Epoch 1/75
5000/5000 [==============================] - 14s 3ms/sample - loss: 0.5237 - val_loss: 0.4498
Epoch 2/75
5000/5000 [==============================] - 2s 393us/sample - loss: 0.4036 - val_loss: 0.3974
Epoch 3/75
5000/5000 [==============================] - 2s 408us/sample - loss: 0.3663 - val_loss: 0.3627
Epoch 4/75
5000/5000 [==============================] - 3s 628us/sample - loss: 0.3515 - val_loss: 0.3523
Epoch 5/75
5000/5000 [==============================] - 3s 652us/sample - loss: 0.3418 - val_loss: 0.3436
Epoch 6/75
5000/5000 [==============================] - 3s 596us/sample - loss: 0.3323 - val_loss: 0.3342
Epoch 7/75
5000/5000 [==============================] - 2s 375us/sample - loss: 0.3244 - val_loss: 0.3252
Epoch 8/75
5000/5000 [==============================] - 2s 400us/sample - loss: 

D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


number of epoch :  75
ami k-means for non encoded data:  0.37889971266005357
nmi k-means for non encoded data:  0.4052825970215828
Train on 5000 samples, validate on 5000 samples
Epoch 1/100
5000/5000 [==============================] - 8s 2ms/sample - loss: 0.5029 - val_loss: 0.4261
Epoch 2/100
5000/5000 [==============================] - 1s 283us/sample - loss: 0.3986 - val_loss: 0.3828
Epoch 3/100
5000/5000 [==============================] - 1s 286us/sample - loss: 0.3644 - val_loss: 0.3570
Epoch 4/100
5000/5000 [==============================] - 1s 280us/sample - loss: 0.3505 - val_loss: 0.3493
Epoch 5/100
5000/5000 [==============================] - 1s 277us/sample - loss: 0.3412 - val_loss: 0.3451
Epoch 6/100
5000/5000 [==============================] - 1s 284us/sample - loss: 0.3336 - val_loss: 0.3356
Epoch 7/100
5000/5000 [==============================] - 1s 279us/sample - loss: 0.3267 - val_loss: 0.3359
Epoch 8/100
5000/5000 [==============================] - 1s 279us/sample -

...

number of epoch :  125
ami k-means for non encoded data:  0.37403247158735703
nmi k-means for non encoded data:  0.4037288147533571
Train on 5000 samples, validate on 5000 samples
Epoch 1/150
5000/5000 [==============================] - 11s 2ms/sample - loss: 0.5060 - val_loss: 0.4204
Epoch 2/150
5000/5000 [==============================] - 2s 467us/sample - loss: 0.3986 - val_loss: 0.3883
Epoch 3/150
5000/5000 [==============================] - 2s 495us/sample - loss: 0.3691 - val_loss: 0.3627
Epoch 4/150
5000/5000 [==============================] - 2s 458us/sample - loss: 0.3512 - val_loss: 0.3511
Epoch 5/150
5000/5000 [==============================] - 2s 400us/sample - loss: 0.3447 - val_loss: 0.3456
Epoch 6/150
5000/5000 [==============================] - 2s 419us/sample - loss: 0.3302 - val_loss: 0.3302
Epoch 7/150
5000/5000 [==============================] - 2s 318us/sample - loss: 0.3223 - val_loss: 0.3251
Epoch 8/150
5000/5000 [==============================] - 2s 355us/sample

5000/5000 [==============================] - 2s 312us/sample - loss: 0.2806 - val_loss: 0.2940
Epoch 76/150
5000/5000 [==============================] - 2s 308us/sample - loss: 0.2798 - val_loss: 0.2927
Epoch 77/150
5000/5000 [==============================] - 2s 331us/sample - loss: 0.2798 - val_loss: 0.2914
Epoch 78/150
5000/5000 [==============================] - 2s 402us/sample - loss: 0.2786 - val_loss: 0.2909
Epoch 79/150
5000/5000 [==============================] - 2s 318us/sample - loss: 0.2785 - val_loss: 0.2914
Epoch 80/150
5000/5000 [==============================] - 2s 315us/sample - loss: 0.2787 - val_loss: 0.2911
Epoch 81/150
5000/5000 [==============================] - 2s 322us/sample - loss: 0.2792 - val_loss: 0.2919
Epoch 82/150
5000/5000 [==============================] - 2s 318us/sample - loss: 0.2782 - val_loss: 0.2907
Epoch 83/150
5000/5000 [==============================] - 2s 311us/sample - loss: 0.2781 - val_loss: 0.2906
Epoch 84/150
5000/5000 [=================

5000/5000 [==============================] - 4s 706us/sample - loss: 0.2694 - val_loss: 0.2878


number of epoch :  150
ami k-means for non encoded data:  0.43117018258615164
nmi k-means for non encoded data:  0.45080429372157793
ami for voting system:  0.27005641030603833
nmo for voting system:  0.2797097632712944


D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureW

| epoch | ami | nmi |
| --- | --- | --- |
| 50 | 0.370 | 0.398|
| 75 | 0.378 |  0.405|
| 100 | 0.415 |  0.438|
| 125 | 0.374 |  0.403|
| 150 | 0.431 |  0.450|

We see immediatly that the results drop drasticly with this results, we can explain this by the fact that maybe 5 clusters si not enough, indeed  we have noticed a lot of situation where there was an aquality leading to a random choice which isn't always a good idea<br>
Let's try 10 autoencoder instead of 5 only with respectively 10,20,30,40,50,60,70,80,90 and epochs for each one of them

In [145]:
epochs=[10,20,30,40,50,60,70,80,90,100]
vote_ensemble(epochs)

Train on 5000 samples, validate on 5000 samples
Epoch 1/10
5000/5000 [==============================] - 11s 2ms/sample - loss: 0.5064 - val_loss: 0.4301
Epoch 2/10
5000/5000 [==============================] - 2s 364us/sample - loss: 0.3996 - val_loss: 0.3862
Epoch 3/10
5000/5000 [==============================] - 2s 386us/sample - loss: 0.3654 - val_loss: 0.3596
Epoch 4/10
5000/5000 [==============================] - 2s 323us/sample - loss: 0.3501 - val_loss: 0.3618
Epoch 5/10
5000/5000 [==============================] - 2s 324us/sample - loss: 0.3428 - val_loss: 0.3412
Epoch 6/10
5000/5000 [==============================] - 2s 326us/sample - loss: 0.3301 - val_loss: 0.3352
Epoch 7/10
5000/5000 [==============================] - 2s 319us/sample - loss: 0.3210 - val_loss: 0.3242
Epoch 8/10
5000/5000 [==============================] - 2s 323us/sample - loss: 0.3153 - val_loss: 0.3195
Epoch 9/10
5000/5000 [==============================] - 2s 326us/sample - loss: 0.3131 - val_loss: 0.3176

D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


number of epoch :  10
ami k-means for non encoded data:  0.41613175424675825
nmi k-means for non encoded data:  0.43021632753538314
Train on 5000 samples, validate on 5000 samples
Epoch 1/20
5000/5000 [==============================] - 11s 2ms/sample - loss: 0.5247 - val_loss: 0.4455
Epoch 2/20
5000/5000 [==============================] - 2s 336us/sample - loss: 0.4035 - val_loss: 0.4002
Epoch 3/20
5000/5000 [==============================] - 2s 335us/sample - loss: 0.3754 - val_loss: 0.3749
Epoch 4/20
5000/5000 [==============================] - 2s 331us/sample - loss: 0.3574 - val_loss: 0.3538
Epoch 5/20
5000/5000 [==============================] - 2s 350us/sample - loss: 0.3444 - val_loss: 0.3456
Epoch 6/20
5000/5000 [==============================] - 2s 328us/sample - loss: 0.3374 - val_loss: 0.3392
Epoch 7/20
5000/5000 [==============================] - 2s 327us/sample - loss: 0.3279 - val_loss: 0.3344
Epoch 8/20
5000/5000 [==============================] - 2s 333us/sample - loss:

D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


number of epoch :  20
ami k-means for non encoded data:  0.37354531370743027
nmi k-means for non encoded data:  0.3878600798487371
Train on 5000 samples, validate on 5000 samples
Epoch 1/30
5000/5000 [==============================] - 12s 2ms/sample - loss: 0.5118 - val_loss: 0.4338
Epoch 2/30
5000/5000 [==============================] - 2s 351us/sample - loss: 0.4019 - val_loss: 0.3781
Epoch 3/30
5000/5000 [==============================] - 2s 482us/sample - loss: 0.3613 - val_loss: 0.3596
Epoch 4/30
5000/5000 [==============================] - 2s 348us/sample - loss: 0.3482 - val_loss: 0.3507
Epoch 5/30
5000/5000 [==============================] - 2s 349us/sample - loss: 0.3379 - val_loss: 0.3350
Epoch 6/30
5000/5000 [==============================] - 2s 344us/sample - loss: 0.3274 - val_loss: 0.3292
Epoch 7/30
5000/5000 [==============================] - 2s 349us/sample - loss: 0.3216 - val_loss: 0.3251
Epoch 8/30
5000/5000 [==============================] - 2s 345us/sample - loss: 

...

D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


number of epoch :  90
ami k-means for non encoded data:  0.3785571371155427
nmi k-means for non encoded data:  0.40329162329454865
Train on 5000 samples, validate on 5000 samples
Epoch 1/100
5000/5000 [==============================] - 13s 3ms/sample - loss: 0.5174 - val_loss: 0.4517
Epoch 2/100
5000/5000 [==============================] - 2s 344us/sample - loss: 0.4048 - val_loss: 0.3887
Epoch 3/100
5000/5000 [==============================] - 2s 343us/sample - loss: 0.3622 - val_loss: 0.3632
Epoch 4/100
5000/5000 [==============================] - 2s 335us/sample - loss: 0.3621 - val_loss: 0.3537
Epoch 5/100
5000/5000 [==============================] - 2s 430us/sample - loss: 0.3417 - val_loss: 0.3469
Epoch 6/100
5000/5000 [==============================] - 2s 343us/sample - loss: 0.3325 - val_loss: 0.3404
Epoch 7/100
5000/5000 [==============================] - 2s 335us/sample - loss: 0.3261 - val_loss: 0.3273
Epoch 8/100
5000/5000 [==============================] - 2s 343us/sample 

5000/5000 [==============================] - 2s 347us/sample - loss: 0.2811 - val_loss: 0.2920
Epoch 76/100
5000/5000 [==============================] - 2s 352us/sample - loss: 0.2804 - val_loss: 0.2919
Epoch 77/100
5000/5000 [==============================] - 2s 350us/sample - loss: 0.2802 - val_loss: 0.2918
Epoch 78/100
5000/5000 [==============================] - 2s 346us/sample - loss: 0.2807 - val_loss: 0.2927
Epoch 79/100
5000/5000 [==============================] - 2s 346us/sample - loss: 0.2804 - val_loss: 0.2924
Epoch 80/100
5000/5000 [==============================] - 2s 349us/sample - loss: 0.2797 - val_loss: 0.2956
Epoch 81/100
5000/5000 [==============================] - 2s 347us/sample - loss: 0.2802 - val_loss: 0.2913
Epoch 82/100
5000/5000 [==============================] - 2s 468us/sample - loss: 0.2793 - val_loss: 0.2915
Epoch 83/100
5000/5000 [==============================] - 2s 348us/sample - loss: 0.2786 - val_loss: 0.2906
Epoch 84/100
5000/5000 [=================

number of epoch :  100
ami k-means for non encoded data:  0.38701158297809723
nmi k-means for non encoded data:  0.4114817702593212
ami for voting system:  0.19680157818598643
nmo for voting system:  0.2184144698507875


D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\annaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureW

| epoch | ami | nmi |
| --- | --- | --- |
| 10 | 0.416|  0.430|
| 20 | 0.373 |  0.387|
| 30 | 0.415 |  0.4395|
| 40 | 0.385 |  0.417|
| 50 | 0.433|  0.457|
| 60 | 0.366 | 0.389|
| 70 | 0.396 |  0.4202|
| 80 | 0.379 |  0.408|
| 90 | 0.378 |  0.403|
| 100 | 0.387 |  0.411|

The results not even didn't improve but they also drop by approximatively 20 point for both ami and nmi.

**The final conclusion would be that despite the fact that the Co-Association matrix ensemble cluster based method do not improve the result over a simple method using a K-means on an encoded data, it didn't drop the results like our voting system did<br>
So the Co-Association matrix method can be considered as a good consensus method for clustering**